
# A widget to reproduce the outputs of Hartmut Zohm's paper, "On the size of tokamak fusion power plants"
## Philosophical Transactions A, 04 February 2019

http://dx.doi.org/10.1098/rsta.2017.0437

Instructions: Execute all the cells, then find the interactive widget at the bottom.

In [1]:
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from copy import copy

In [263]:
class ReactorScaling:
    
    def __init__(self, A, q, H, beta_N, f_GW, f_LH, P_fus, Q_CD, Q_PB):
        self.A = A
        self.q = q
        self.H = H
        self.beta_N = beta_N
        self.f_GW = f_GW
        self.f_LH = f_LH
        self.P_fus = P_fus
        self.Q_CD = Q_CD
        self.Q_PB = Q_PB
        
    def set_B_and_R_ref(self, B_ref, R_ref):
        self.B = B_ref
        self.R = R_ref


Here are ITER's specifications as reported in the paper. 

Note that there is an uncertainty as to which value of $Q_\mathrm{CD}$ Zohm assumed, since ITER does not have a self-sustaining current drive.

In [264]:
RS_ITER = ReactorScaling(A=3.1, q=3.1, H=1.0, beta_N=1.8, f_GW=0.85, f_LH=1.33, P_fus=400, Q_CD=1, Q_PB=10)
RS_ITER.set_B_and_R_ref(5.2, 6.2)

$P_\mathrm{fus}$ is the fusion power produced in the reactor.

$$P_\mathrm{fus} = c_\mathrm{fus} \frac{\beta_N^2 B^4 A^3}{q^2 A^4} \qquad \textrm{(Eq. 2.1)}$$
Use ITER's specifications we can solve for $c_\mathrm{fus}$, then, 
solve for $B$ in terms of the other variables. 
$$B = \frac{P_\mathrm{fus}^{1/4} q^{1/2} A}{c_\mathrm{fus}^{1/4} \beta_N^{1/2} R^{3/4}} $$


In [265]:
def B_Pfus(R, reactor, reactor_ref):
    """Equation 2.1, solved for B.

    """
    P_fus_ref = reactor_ref.P_fus
    A_ref     = reactor_ref.A
    q_ref     = reactor_ref.q
    beta_N_ref= reactor_ref.beta_N
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    P_fus     = reactor.P_fus
    q         = reactor.q
    A         = reactor.A
    beta_N    = reactor.beta_N
    
    c_fus = P_fus_ref * q_ref**2 * A_ref**4 / (beta_N_ref**2 * B_ref**4 * R_ref**3)
    numerator = P_fus**0.25 * q**0.5 * A
    denominator = c_fus ** 0.25 * beta_N**0.5 * R**(0.75)
    return numerator/denominator

$Q_{\mathrm{PB}}$ is the ratio of the fusion power to the auxiliary power needed to sustain the plasma,

$$ Q_{\mathrm{PB}} = \frac{P_\mathrm{fus}}{P_\mathrm{AUX}} = \frac{1}{c_\mathrm{PB}\left(q^{3.1} A^{3.53} H^{-3.23} \beta_N^{-0.1} R^{-2.7} B^{-3.7} - 1/5\right)} \qquad \textrm{(Eq. 2.1)}$$
The $1/5$ there is because $1/5$ of the fusion power is in the form of the $\alpha$ particles which heat the plasma.
Using the typical ITER value of $Q_\mathrm{PB} = 10$ we can solve for $c_\mathrm{PB}$, then solve for $B$.

$$B = \left(\frac{c_\mathrm{PB}}{Q_\mathrm{PB}^{-1} + 1/5}\right)^{1/3.7} \frac{q^{1/3.7} A^{3.53/3.7}}{H^{3.23/3.7}\beta_N^{0.1/3.7}R^{2.7/3.7}}$$

In [266]:
def B_Q_PB(R, reactor, reactor_ref):
    """Equation 2.2, solved for B"""
    Q_PB_ref  = reactor_ref.Q_PB
    H_ref     = reactor_ref.H
    A_ref     = reactor_ref.A
    q_ref     = reactor_ref.q
    beta_N_ref= reactor_ref.beta_N
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    Q_PB      = reactor.Q_PB
    q         = reactor.q
    A         = reactor.A
    H         = reactor.H
    beta_N    = reactor.beta_N
    
    c_PB = (Q_PB_ref**(-1) + 1/5) / (q_ref**3.1 * A_ref**3.53 / \
                                     (H_ref**3.23 * beta_N_ref**0.1 * R_ref**2.7 * B_ref**3.7))
    
    first_term = (c_PB / (Q_PB**(-1) + 1/5))**(1/3.7)
    numerator = q**(3.1/3.7) * A**(3.53/3.7)
    denominator = H**(3.23/3.7) * beta_N**(0.1/3.7) * R**(2.7/3.7)
    return first_term * numerator / denominator

$Q_\mathrm{CD}$ is the ratio of fusion power to the current drive power needed to sustain the plasma current. The paper assumes that the tokamak operates in steady state: if the requirement is not met (as for ITER itself), that means the tokamak can only operate in a pulsed mode.

$$ Q_\mathrm{CD} = \frac{P_\mathrm{fus}}{P_\mathrm{CD}} = c_\mathrm{CD} \frac{\beta_N^{3} R^3 B^3}{A^3 f_\mathrm{GW}^2\left( 5 + Z_\mathrm{eff}\right)\left(1 - c_\mathrm{BS} A^{1/2} q \beta_N\right)}$$

I'm not certain about the value of $c_\mathrm{BS}$. From Zohm's 2009 paper it seems plausible that it is $c_\mathrm{BS} = 0.0318 * 0.7 = 0.02226$.

The paper goes not give a value for $Z_\mathrm{eff}$ so we assume that it is unchanged from the ITER value. We can solve for B as a function of the other variables, scaled from the reference (ITER) value:

$$ B = B_\mathrm{ref} \left(\frac{Q_\mathrm{CD}}{Q_\mathrm{CD, ref}}\right)^{1/3}
\left(\frac{A}{A_\mathrm{ref}}\right)
\left(\frac{R_\mathrm{ref}}{R}\right)
\left(\frac{\beta_{N,\mathrm{ref}}}{\beta_N}\right)
\left(\frac{f_\mathrm{GW}}{f_{\mathrm{GW,ref}}}\right)^{2/3}
\left(\frac{1 - c_\mathrm{BS} A^{1/2} q \beta_N}{1 - c_\mathrm{BS} A^{1/2}_\mathrm{ref} q_\mathrm{ref} \beta_{N,\mathrm{ref}}}\right)^{1/3}
$$

In [267]:
def B_Q_CD(R, reactor, reactor_ref):
    """Equation 2.3, solved for B"""
    
    Q_CD_ref  = reactor_ref.Q_CD
    f_GW_ref  = reactor_ref.f_GW
    A_ref     = reactor_ref.A
    q_ref     = reactor_ref.q
    beta_N_ref= reactor_ref.beta_N
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    Q_CD      = reactor.Q_CD
    f_GW      = reactor.f_GW
    q         = reactor.q
    A         = reactor.A
    beta_N    = reactor.beta_N
    
    Z_eff_ref = 1 # unsure what these are, so they are treated as unchanged from the ITER values.
    Z_eff = Z_eff_ref 
    c_BS = 0.0318 * 0.7
    return B_ref * (Q_CD / Q_CD_ref)**(1/3) * (A/A_ref) * (R_ref / R) * \
        (beta_N_ref / beta_N) * (f_GW / f_GW_ref) **(2/3) * ((5 + Z_eff)/(5 + Z_eff_ref))**(1/3) * \
        ((1 - c_BS * A**0.5 * q * beta_N) / (1 - c_BS * A_ref**0.5 * q_ref * beta_N_ref))**(1/3)

"For a fully-consistent solution, $Q_{CD} = Q_{PB}$ but as the two quantities assume large values, their difference will not influence the power balance strongly."

$f_{z, \mathrm{div}}$ represents the concentration of impurities in the divertor. It's not explicitly calculated in Zohm's paper; instead, the equation is scaled relative to the (non-calculated) ITER value.

$$f_{Z, \mathrm{div}} = c_\mathrm{exh} \frac{f_{\mathrm{LH}}^{1.14}q^{0.32} B^{0.88} R^{1.33}}{f_\mathrm{GW}^{1.18}} \qquad \textrm{Eq. 2.4}$$

The scaling equation relative to the (ITER) reference point is
$$ B = B_\mathrm{ref} \left(\frac{f_\mathrm{GW}}{f_{\mathrm{GW},\mathrm{ref}}}\right)^{1.18/0.88}
\left(\frac{f_{\mathrm{LH},\mathrm{ref}}}{f_\mathrm{LH}}\right)^{1.14/0.88}
\left(\frac{q_\mathrm{ref}}{q}\right)^{0.32/0.88}
\left(\frac{R_\mathrm{ref}}{R}\right)^{1.33/0.88}
$$

In [268]:
def B_f_zdiv(R, reactor, reactor_ref):
    """Equation 2.4 solved for B"""
    
    f_GW_ref  = reactor_ref.f_GW
    f_LH_ref  = reactor_ref.f_LH
    q_ref     = reactor_ref.q
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    f_GW      = reactor.f_GW
    f_LH      = reactor.f_LH
    q         = reactor.q
    
    B = B_ref * (f_GW / f_GW_ref)**(1.18/0.88) * (f_LH_ref/f_LH)**(1.14/0.88) * \
         (q_ref / q)**(0.32/0.88) * (R_ref / R)**(1.33/0.88)
    return B

## Make an interactive plot

Plot the required $B$ as a function of $R$, depending on the reactor parameters.

In [278]:
R = np.arange(2, 10.1, 0.25)

def f(my_P_fus, my_Q_PB, my_Q_CD, my_f_LH, my_f_GW, my_q, my_H, my_beta_N):
    reactor= ReactorScaling(3.1, my_q, my_H, my_beta_N, my_f_GW, my_f_LH, my_P_fus, my_Q_CD, my_Q_PB)
    bs_pfus = B_Pfus(rs, reactor, RS_ITER)
    bs_QPB  = B_Q_PB(rs, reactor, RS_ITER)
    bs_qcd = B_Q_CD(rs, reactor, RS_ITER)
    bs_fz = B_f_zdiv(rs, reactor, RS_ITER)
    
    
    fig, ax = plt.subplots()
    plt.rcParams['font.size'] = 18
    ax.plot(rs, bs_pfus, label='Pfus', color='red')
    ax.plot(rs, bs_QPB, label='Q_PB', color='blue')
    ax.plot(rs, bs_qcd, label='Q_CD', color='green')
    ax.plot(rs, bs_fz, label='fz', color='yellow')

    plt.ylim(5, 14)
    plt.xlim(2,10)
    plt.xlabel(r'$R/\mathrm{m}$')
    plt.ylabel(r'$B/\mathrm{T}$')

    plt.show()

interactive_plot = interactive(f, my_f_LH=widgets.FloatSlider(description=r'$f_\mathrm{LH}$', min=1.0, max=1.33, step=0.01, value=1.33),
                               my_f_GW=widgets.FloatSlider(description=r'$f_\mathrm{GW}$', min=0.85, max=1.2, step=0.05, value=0.85),
                               my_q=widgets.FloatSlider(description=r'$q$', min=3.1, max=6, step=0.1, value=3.1),
                               my_H=widgets.FloatSlider(description=r'$H$', min=1.0, max=1.2, step=0.1, value=1.0),
                               my_P_fus=widgets.IntSlider(description=r'$P_\mathrm{fus}$', min=400, max=3500, step=100, value=400),
                               my_Q_PB=widgets.IntSlider(description=r'$Q_\mathrm{PB}$', min=1, max=100, step=1, value=10), 
                               my_Q_CD=widgets.IntSlider(description=r'$Q_\mathrm{CD}$', min=1, max=100, step=1, value=10),
                               my_beta_N=widgets.FloatSlider(description=r'$\beta_\mathrm{N}$', min=1.8, max=3.5, step=0.1, value=1.8))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=400, description='$P_\\mathrm{fus}$', max=3500, min=400, step=100), IntS…

The default settings for the plot reproduce Figure 1.
Try changing the sliders to the values for the other figures!